In [142]:
# ======================================================================================================
# 
# ------------------------------------------ Starting HERE ---------------------------------------------
# 
# ======================================================================================================

import carla
import math
import time
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont

In [2]:
# Get access to the client and the world
client = carla.Client('localhost', 2000)
world = client.get_world()
world.unload_map_layer(carla.MapLayer.All)

In [80]:
# Define the type of car we want to use
vehicle_blueprint = client.get_world().get_blueprint_library().filter('model3')[0]

In [161]:
# ====================================
# -- SPAWN VEHICLE -------------------
# ====================================
spawn_point = client.get_world().get_map().get_spawn_points()[23] # works
vehicle = client.get_world().spawn_actor(vehicle_blueprint, spawn_point)

In [153]:
# set spectator to be on the car's dash
spectator = world.get_spectator()
spec_transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(spec_transform)

In [162]:
# Add RGB camera
camera_bp = client.get_world().get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('sensor_tick', '0.2') # take a picture every 1 second(s)

camera_init_trans = carla.Transform(carla.Location(x=1.75, z=1))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [163]:
def transformed_frame(frame):
    lower_right = (625, 396)
    upper_right = (427, 288)
    lower_left = (14,394)
    upper_left = (215,284)

    src = np.float32([upper_left, lower_left, upper_right, lower_right])
    dst = np.float32([[0, 0], [0, 480], [640, 0], [640, 480]])

    matrix = cv2.getPerspectiveTransform(src, dst)
    transformed_frame = cv2.warpPerspective(frame, matrix, (640,480))
    return transformed_frame

def masking(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    l = np.array([0,0,200])
    u = np.array([255,50,255])
    mask = cv2.inRange(hsv, l, u)
    return mask

def threshold(frame, mask):
    result = cv2.bitwise_and(frame, frame, mask=mask)
    return result

def midlane_coordinates(frame):
    histogram = np.sum(frame[240:, :], axis =0)
    midpoint = int(histogram.shape[0]/2)
    left_x = np.argmax(histogram[:midpoint,2])
    right_x = np.argmax(histogram[midpoint:,2]) + midpoint

    mid_value = (histogram[left_x,2] + histogram[right_x,2]) / 2
    if (mid_value > 30000):
    
        return (left_x + right_x) / 2

    return -1

def pipeline(filename):
    img = cv2.imread(filename)
    frame = cv2.resize(img, (640,480))
    frame = transformed_frame(frame)
    mask = masking(frame)
    frame = threshold(frame,mask)
    mid = midlane_coordinates(frame)
    return mid

def sliding_window(frame, left_l, right_l):
    #contours is the shifting of white pixels
    lx = []
    rx = []
    y = 479

    while y > 0:
        img = frame[y-40:y, left_l-50:left_l+50]
        contours,h = cv2.findContours(img, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        for c in contours:
            m = cv2.moments(c)
            if(m["m00"] != 0):
                cx = int(m["m10"] / m["m01"]) #center of the contours
                left_l = left_l - 50 + cx
                lx.append(left_l)

        img = frame[y-40:y, right_l-50:right_l+50]
        contours,h = cv2.findContours(img, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        for c in contours:
            m = cv2.moments(c)
            if(m["m00"] != 0):
                cx = int(m["m10"] / m["m00"]) #center of the contours
                right_l = right_l - 50 + cx
                rx.append(right_l)
        midx = int((right_l+left_l)/2)

        cv2.rectangle(frame, (midx-25,y), (midx+25,y-40), (255,255,255), 1)
        y = y - 40
    return frame

def midlane_draw(frame, mid):
    img = np.zeros((512,512,3), np.uint8)
    cv2.line(img,(0,0),(511,511),(255,0,0),5)
    frame = img
    return frame

In [164]:
# used to store midpoint associated with each image
# so we can label the images after the fact
camera_results = {}

In [166]:
# ====================================
# -- LANE FOLLOWING ------------------
# ====================================

# this is a callback function to be applied to camera.listen
# -- each time it captures an image, it will compute the 
# -- direction and control the car
def compute_direction(image, vehicle, camera_results):
    image.save_to_disk('test07/%86d.png' % image.frame)

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    img[:,:,3] = 255
        
    frame = cv2.resize(img, (640,480))
    frame = transformed_frame(frame)
    mask = masking(frame)
    frame = threshold(frame,mask)
    mid = midlane_coordinates(frame)
    mid = int(mid)
    print(f'mid: {mid}')
    
    # Apply control
    control = carla.VehicleControl()
    control.throttle = 0.3
    
    camera_results[image.frame] = mid

    if mid == -1:
        control.throttle = 0.0
        control.brake = 0.2
        vehicle.apply_control(control)    
        return
    if mid > 321:
        # go right
        control.steer = 0.01
    elif mid < 321:
        # go left
        control.steer = -0.01
    else:
        # go straight
        control.steer = 0.0
    
    # apply control
    vehicle.apply_control(control)

In [167]:
# Start the camera + control
camera.listen(lambda image: compute_direction(image, vehicle, camera_results))

mid: 290
mid: 290
mid: 290
mid: 290
mid: 290
mid: 290
mid: 290
mid: 290
mid: 290
mid: 291
mid: 290
mid: 292
mid: 294
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: 316
mid: 319
mid: 317
mid: 319
mid: 321
mid: 319
mid: 319
mid: 322
mid: 325
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: 343
mid: 343
mid: 342
mid: 339
mid: 338
mid: 335
mid: 335
mid: 332
mid: 327
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: 336
mid: 334
mid: 332
mid: 329
mid: 326
mid: 323
mid: 320
mid: 316
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: 323
mid: 322
mid: 320
mid: 318
mid: 319
mid: 320
mid: 320
mid: 323
mid: 323
mid: 318
mid: -1
mid: -1
mid: 326
mid: 325
mid: 323
mid: 318
mid: 317
mid: 319
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: 341
mid: 339
mid: 338
mid: 336
mid: 333
mid: 330
mid: 326
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: -1
mid: 327
mid: 324
mid: 322
mid: 320
mid: 319
mid: 318
mid: 318
mid: 319
mid: 320
mid: 322
mid: 322
mid: 31

In [168]:
camera.stop()

mid: -1


In [169]:
# Label the images
for frame, mid in camera_results.items():
    img = Image.open('test07/%86d.png' % frame)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype("arial.ttf", size=50)
    text_position = (50, 50)  # example position
    text = ""
    
    if mid == -1:
        text = "brake"
        draw.text(text_position, text, fill=(255, 0, 0), font=font)
        img.save('test07-words/%86d.png' % frame)
        continue

    if mid > 321:
        text = "go right"
        draw.text(text_position, text, fill=(0, 0, 0), font=font)
        img.save('test07-words/%86d.png' % frame)
    
    elif mid < 321:
        text = "go left"
        draw.text(text_position, text, fill=(0, 0, 0), font=font)
        img.save('test07-words/%86d.png' % frame)
    
    else: # mid == 321
        text = "go straight"
        draw.text(text_position, text, fill=(0, 255, 0), font=font)
        img.save('test07-words/%86d.png' % frame)

In [160]:
# for clean-up
vehicle.destroy()

if camera:
    camera.destroy()

In [12]:
# ======================================================================================================
# 
# ------------------------------ Additional helper methods ---------------------------------------------
#
# ======================================================================================================
# waypoints are from 0-220 incrementing by 4; each waypoint is +1 in the x location

def draw_waypoints(waypoints, road_id=None, life_time=50.0):
    for waypoint in waypoints:
        # if we only want to draw waypoints on a specific road
        if road_id:
            if waypoint.road_id == road_id:
                # world.debug.draw_string(waypoint.transform.location, 'O', draw_shadow=False,color=carla.Color(r=0, g=255, b=0), life_time=life_time, persistent_lines=True)
                world.debug.draw_string(waypoint.transform.location, f'{waypoint.road_id}', draw_shadow=False,color=carla.Color(r=0, g=255, b=0), life_time=life_time, persistent_lines=True)
        # draw all waypoints
        else:
            world.debug.draw_string(waypoint.transform.location, 'O', draw_shadow=False,color=carla.Color(r=0, g=255, b=0), life_time=life_time, persistent_lines=True)
            
# examples usage:
waypoints = client.get_world().get_map().generate_waypoints(distance=1.0)
draw_waypoints(waypoints, road_id=None, life_time=20)

# currently on road 18, want to get to road 14

In [13]:
# filter waypoints by road_id and provide specific indexing for them for access
filtered_waypoints = []
for waypoint in waypoints:
    if waypoint.road_id == 1:
        filtered_waypoints.append(waypoint)
        
for i, sp in enumerate(filtered_waypoints):
    world.debug.draw_string(sp.transform.location, str(i), draw_shadow=False, color=carla.Color(r=0, g=255, b=0), life_time=20, persistent_lines=True)

In [29]:
# show spawn points
test_spawn = client.get_world().get_map().get_spawn_points()

for i, sp in enumerate(test_spawn):
    world.debug.draw_string(sp.location, str(i), draw_shadow=False, color=carla.Color(r=255,g=0,b=0), life_time=20, persistent_lines=True)

In [170]:
# Compile the images into a video
import os

image_folder = 'test07-words'
video_name = 'lane_following.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# video name, fourcc, fps, tuple of dimensions
video = cv2.VideoWriter(video_name, 0, 5, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()
